# Pipeline Completo: Tópicos e Persistência (MVP - Semana 3)

Este notebook cumpre os requisitos da Semana 3 do projeto de Big Data. O objetivo é executar o pipeline completo de engenharia de dados:

1.  **Carregar** os dados brutos da Semana 1.
2.  **Limpar** os textos.
3.  **Classificar Sentimento** (lógica da Semana 2).
4.  **Extrair Tópicos** (nova etapa da Semana 3).
5.  **Salvar no Banco de Dados** (nova etapa da Semana 3).

Todo o código de processamento é importado do módulo `src/processing/text_processor.py` para manter este notebook limpo e modular.

### Passo 0: Instalação de Dependências

Esta célula instala todas as bibliotecas Python necessárias para o pipeline completo.

* `leia-br`: Para análise de sentimento.
* `unidecode`: Para normalização de texto.
* `spacy`: Para extração de tópicos (NLP avançado).
* `duckdb`: Para o banco de dados em arquivo.
* `sqlalchemy`: Interface para salvar DataFrames no DuckDB.

**Importante:** Após executar esta célula, **reinicie o kernel** do seu Jupyter para que as novas bibliotecas sejam carregadas.

In [1]:
# Instala todas as bibliotecas necessárias para o pipeline
%pip install leia-br unidecode spacy duckdb sqlalchemy duckdb-engine

# Baixa o modelo de linguagem "pt_core_news_sm" (pequeno e eficiente)
!python -m spacy download pt_core_news_sm

# Lembre-se de REINICIAR O KERNEL do seu notebook após a instalação!

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     --------------------------------------- 0.0/13.0 MB 393.8 kB/s eta 0:00:33
      --------------------------------------- 0.3/13.0 MB 2.1 MB/s eta 0:00:06
     -- ------------------------------------- 0.7/13.0 MB 4.3 MB/s eta 0:00:03
     --- ------------------------------------ 1.3/13.0 MB 5.8 MB/s eta 0:00:03
     ----- ---------------------------------- 1.8/13.0 MB 6.9 MB/s eta 0:00:02
     ------- -------------------------------- 2.4/13.0 MB 7.7 MB/s eta 0:00:02
     --------- ------------------------------ 2.9/13.0 MB 8.2 MB/s eta 0:00:02
     ---------- ----------------------------- 3.5/13.0 MB 8.6 MB/s eta 0:00:02
     ------------ --------------------------- 4.1/13.0 MB 8.9 MB/s eta 0:00:02
     -------------- ------------------------- 4.6/13.0 MB 9.2 MB/s eta 0:00:01
     --------------- ------------------------ 5.2/13.0 MB 9.4 MB/


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Passo 1: Setup e Carregamento dos Dados

Configuramos os caminhos relativos do projeto para que o notebook possa encontrar tanto o módulo `src` quanto os arquivos de dados na pasta `data`, independentemente da máquina onde é executado.

In [2]:
import pandas as pd
import sys
import os

# --- Lógica para encontrar os diretórios do projeto ---
try:
    notebook_dir = os.path.dirname(os.path.abspath("__file__"))
except NameError:
    notebook_dir = os.getcwd()

project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
src_path = os.path.join(project_root, 'src')

if src_path not in sys.path:
    sys.path.insert(0, src_path)
    
print(f"Diretório raiz do projeto encontrado em: {project_root}")
print(f"Diretório 'src' adicionado ao path: {src_path}")

# --- Importação das nossas funções customizadas ---
try:
    from processing.text_processor import (
        limpar_texto, 
        classificar_sentimento, 
        extrair_topicos, 
        salvar_no_banco_de_dados
    )
    print("Módulos de processamento importados com sucesso!")
except ImportError as e:
    print(f"ERRO NA IMPORTAÇÃO: {e}")
    
# --- Carregamento do Arquivo com Caminho Relativo ---
file_path = os.path.join(project_root, 'data', 'raw', 'comentarios_produtos.csv')

print(f"\nTentando carregar o arquivo de: {file_path}")
try:
    df_reviews = pd.read_csv(file_path, sep='|', on_bad_lines='skip')
    print("Arquivo de comentários carregado com sucesso!")
    display(df_reviews.head())
except Exception as e:
    print(f"\nOcorreu um erro ao carregar o arquivo: {e}")

Diretório raiz do projeto encontrado em: C:\Users\Teste\Documents\Gustavo\Trabalhos 5° Ano\Business Inteligence & Big Data\Par Temático 2 - Big Data\bi-web-scrapping
Diretório 'src' adicionado ao path: C:\Users\Teste\Documents\Gustavo\Trabalhos 5° Ano\Business Inteligence & Big Data\Par Temático 2 - Big Data\bi-web-scrapping\src
Módulos de processamento importados com sucesso!

Tentando carregar o arquivo de: C:\Users\Teste\Documents\Gustavo\Trabalhos 5° Ano\Business Inteligence & Big Data\Par Temático 2 - Big Data\bi-web-scrapping\data\raw\comentarios_produtos.csv
Arquivo de comentários carregado com sucesso!


,id_produto,id_avaliacoes,id_comentario,rating_do_comentario,quantidade_avaliacoes_do_produto,comentario_about_produto,data_coleta
0,80054742,80054742,68ea88d327ae04a3a6680c51,5,2308,O smartphone parece ser muito bom apesar de ai...,2025-10-22 21:16:16
1,80054742,80054742,68ac9172bbb755975d29447a,5,2308,Achei uma ótima escolha. Muito útil para traba...,2025-10-22 21:16:16
2,80054742,80054742,68b60274312495cfd879bd61,5,2308,Muito bom e entrega super rápida no mesmo dia.,2025-10-22 21:16:16
3,80054742,80054742,68ac9190bbb755975d29ab31,5,2308,Excelente produto.,2025-10-22 21:16:16
4,80054742,80054742,68ac9172bbb755975d293822,4,2308,Ótimo,2025-10-22 21:16:16


### Passo 2: Limpeza e Pré-processamento do Texto

Aplicamos a função `limpar_texto` do nosso módulo para normalizar os comentários, removendo acentos, pontuação e convertendo para minúsculas.

In [3]:
print("Iniciando limpeza e pré-processamento do texto...")

df_cleaned = df_reviews.copy()
df_cleaned.dropna(subset=['comentario_about_produto'], inplace=True)

# Aplica a função de limpeza importada
df_cleaned['comentario_limpo'] = df_cleaned['comentario_about_produto'].apply(limpar_texto)

# Remove linhas que ficaram vazias após a limpeza
df_cleaned = df_cleaned[df_cleaned['comentario_limpo'].str.strip() != '']

print("Limpeza de texto concluída.")
display(df_cleaned[['comentario_about_produto', 'comentario_limpo']].head())

Iniciando limpeza e pré-processamento do texto...
Limpeza de texto concluída.


,comentario_about_produto,comentario_limpo
0,O smartphone parece ser muito bom apesar de ai...,o smartphone parece ser muito bom apesar de ai...
1,Achei uma ótima escolha. Muito útil para traba...,achei uma otima escolha muito util para trabal...
2,Muito bom e entrega super rápida no mesmo dia.,muito bom e entrega super rapida no mesmo dia
3,Excelente produto.,excelente produto
4,Ótimo,otimo


### Passo 3: Classificação de Sentimento (Lógica da Semana 2)

Executamos a função `classificar_sentimento` do nosso módulo, que aplica a robusta lógica híbrida (baseada em notas e análise de texto) para classificar cada review.

In [4]:
print("Iniciando a classificação de sentimento...")

# Aplica a função de classificação completa importada
df_classificado = classificar_sentimento(df_cleaned)

print("\nAnálise de sentimento concluída!")
display(df_classificado['sentimento'].value_counts())

Iniciando a classificação de sentimento...

Análise de sentimento concluída!


sentimento
Positivo    2162
Negativo      43
Neutro         5
Name: count, dtype: int64

### Passo 4: Extração de Tópicos (Nova Etapa - Semana 3)

Esta é a primeira nova entrega da Semana 3. Usamos a função `extrair_topicos` (que utiliza `spaCy`) para processar o texto limpo e identificar os substantivos-chave, nos dizendo *sobre o que* o cliente está falando.

In [5]:
print("Iniciando a extração de tópicos...")

# Aplica a função de extração de tópicos na coluna de texto limpo
df_com_topicos = df_classificado.copy()
df_com_topicos['topicos'] = df_com_topicos['comentario_limpo'].apply(extrair_topicos)

print("Extração de tópicos concluída.")
display(df_com_topicos[['sentimento', 'topicos', 'comentario_about_produto']].head())

Iniciando a extração de tópicos...
Extração de tópicos concluída.


,sentimento,topicos,comentario_about_produto
0,Positivo,[smartphone],O smartphone parece ser muito bom apesar de ai...
1,Positivo,"[velocidade, armazenamento, proprio, escolha, ...",Achei uma ótima escolha. Muito útil para traba...
2,Positivo,"[dia, entrega]",Muito bom e entrega super rápida no mesmo dia.
3,Positivo,[produto],Excelente produto.
4,Positivo,[],Ótimo


### Passo 5: Persistência no Banco de Dados (Nova Etapa - Semana 3)

Esta é a segunda entrega da Semana 3. Em vez de salvar em um CSV, usamos a função `salvar_no_banco_de_dados` para persistir o DataFrame final em um arquivo **DuckDB**. Este arquivo é a nossa "tabela de fatos" pronta para ser consumida pelo Tableau.

In [6]:
# 1. Validação: exibe os primeiros 20 registros do DataFrame final
print("Exibindo os primeiros 20 registros do DataFrame final para validação:")
display(df_com_topicos[['rating_do_comentario', 'sentimento', 'topicos', 'comentario_about_produto']].head(20))

# 2. Constrói o caminho de saída para o arquivo do banco de dados
db_path_absolute = os.path.join(project_root, 'data', 'output', 'reviews.duckdb')
db_path_relative = os.path.relpath(db_path_absolute, project_root).replace('\\', '/')

# 3. Seleciona as colunas finais para salvar
colunas_para_salvar = [
    'id_produto', 'id_avaliacoes', 'id_comentario', 'rating_do_comentario',
    'comentario_about_produto', 'sentimento', 'topicos', 'data_coleta'
]
df_para_salvar = df_com_topicos[colunas_para_salvar]

# 4. Salva no banco de dados
print(f"\nIniciando gravação no banco de dados em: {db_path_relative}...")
sucesso, mensagem = salvar_no_banco_de_dados(df_para_salvar, db_path_absolute, "reviews_classificadas")

if sucesso:
    print(f"Sucesso! {mensagem} linhas salvas na tabela 'reviews_classificadas'.")
else:
    print(f"Ocorreu um erro ao salvar no DuckDB: {mensagem}")

Exibindo os primeiros 20 registros do DataFrame final para validação:


,rating_do_comentario,sentimento,topicos,comentario_about_produto
0,5,Positivo,[smartphone],O smartphone parece ser muito bom apesar de ai...
1,5,Positivo,"[velocidade, armazenamento, proprio, escolha, ...",Achei uma ótima escolha. Muito útil para traba...
2,5,Positivo,"[dia, entrega]",Muito bom e entrega super rápida no mesmo dia.
3,5,Positivo,[produto],Excelente produto.
4,4,Positivo,[],Ótimo
5,5,Positivo,[],Muito bom
6,5,Positivo,"[marca, aparelhos, recomendo, qualidade, samsung]",Samsung é uma ótima marca de qualidade e eu se...
7,4,Positivo,[produto],O produto é bom
8,5,Positivo,"[produto, mae]","Excelente produto, comprei para minha mãe e el..."
9,5,Positivo,"[marca, gosto, problema, otimo, imagem, preco]","Chegou adiantado, ele é muito lindo, boa image..."



Iniciando gravação no banco de dados em: data/output/reviews.duckdb...


C:\Users\Teste\Documents\Gustavo\Trabalhos 5° Ano\Business Inteligence & Big Data\Par Temático 2 - Big Data\bi-web-scrapping\venv\Lib\site-packages\duckdb_engine\__init__.py:184: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


Sucesso! 2210 linhas salvas na tabela 'reviews_classificadas'.
